# Liftover Nott's
This notebook documents the codes used to lift over nott from hg19 to hg38
These codes are ran in the python console directly for the enviorment is not setup yet.

In [5]:
import pandas as pd


In [3]:

# Cell
import gzip
from liftover import get_lifter


ModuleNotFoundError: No module named 'liftover'

In [9]:
from liftover import ChainFile

class Liftover:
    def __init__(self,fr='hg19',to='hg38',file ='~/hg19ToHg38.over.chain'):
        self.fr,self.to = fr,to
        self.chainmap = ChainFile(file, fr,to)
        #self.fasta
    def variants_liftover(self,chrom,pos):
        if len(chrom) == 1:
            chrom = [chrom]*len(pos)
        lchr,lpos = [],[]
        for c,p in zip(chrom,pos):
            new_c,new_p = self.chrpos_liftover(c,p)
            lchr.append(new_c)
            lpos.append(new_p)
        return lchr,lpos
    def chrpos_liftover(self,chrom,pos):
        try:
            if str(chrom) in ['X','chrX','23']:
                new_c,new_p,_ = self.chainmap['X'][pos][0]
                return 23,new_p
            elif str(chrom) in ['Y','chrY','24']:
                new_c,new_p,_ = self.chainmap['Y'][pos][0]
                return 24,new_p
            elif str(chrom) in ['M','chrM','25','MT']:
                new_c,new_p,_ = self.chainmap['M'][pos][0]
                return 25,new_p
            else:
                new_c,new_p,_ = self.chainmap[int(chrom)][pos][0]
            return int(new_c[3:]),new_p
        except:
            return 0,0
    def region_liftover(self,region):
        imp_cs,imp_start = self.chrpos_liftover(region[0],region[1])
        imp_ce,imp_end = self.chrpos_liftover(region[0],region[2])
        if imp_cs !=imp_ce:
            raise ValueError('After liftover, the region is not in the same chromosome anymore.')
        return imp_cs,imp_start,imp_end


ModuleNotFoundError: No module named 'liftover'

In [1]:
lf = Liftover('hg19','hg38',"hg19ToHg38.over.chain")

NameError: name 'Liftover' is not defined

## MicroGlia lift over

In [ ]:
micro_enh = pd.read_excel("/sc/arion/projects/load/users/patelt16/projects/regulatory_gwas/20230724.rare_prom_enh/wgs.reg.gwas/input/aay0793-nott-table-s5.xlsx",
                          sheet_name = "Microglia enhancers",names = ["chr","start","end"], skiprows = 2 )

In [ ]:
micro_enh = pd.read_excel("/sc/arion/projects/load/users/patelt16/projects/regulatory_gwas/20230724.rare_prom_enh/wgs.reg.gwas/input/aay0793-nott-table-s5.xlsx",
                          sheet_name = "Microglia enhancers",names = ["chr","start","end"], skiprows = 1 )
micro_enh_start = micro_enh.apply(lambda x:  lf.chrpos_liftover(x["chr"].replace("chr",""),x["start"]),axis = 1 )
micro_enh_end = micro_enh.apply(lambda x:  lf.chrpos_liftover(x["chr"].replace("chr",""),x["end"]),axis = 1 )
micro_enh["start"] =  [x[1] for x in micro_enh_start]
micro_enh["end"] =  [x[1] for x in micro_enh_end]
micro_enh.to_csv("//sc/arion/projects/load/users/sunh14/annotation_interaction_gwas/microglia_hg38.Enh.Nott.tsv","\t")

In [ ]:
micro_prom = pd.read_excel("/sc/arion/projects/load/users/patelt16/projects/regulatory_gwas/20230724.rare_prom_enh/wgs.reg.gwas/input/aay0793-nott-table-s5.xlsx",
                          sheet_name = "Microglia promoters",names = ["chr","start","end"], skiprows = 1 )
micro_prom_start = micro_prom.apply(lambda x:  lf.chrpos_liftover(x["chr"].replace("chr",""),x["start"]),axis = 1 )
micro_prom_end = micro_prom.apply(lambda x:  lf.chrpos_liftover(x["chr"].replace("chr",""),x["end"]),axis = 1 )
micro_prom["start"] =  [x[1] for x in micro_prom_start]
micro_prom["end"] =  [x[1] for x in micro_prom_end]
micro_prom.to_csv("//sc/arion/projects/load/users/sunh14/annotation_interaction_gwas/microglia_hg38.Prom.Nott.tsv","\t")

## Astrocyte enhancers liftover for verification

In [ ]:
Ast_enh = pd.read_excel("/sc/arion/projects/load/users/patelt16/projects/regulatory_gwas/20230724.rare_prom_enh/wgs.reg.gwas/input/aay0793-nott-table-s5.xlsx",
                          sheet_name = "Astrocyte enhancers",names = ["chr","start","end"], skiprows = 1 )
Ast_enh_start = Ast_enh.apply(lambda x:  lf.chrpos_liftover(x["chr"].replace("chr",""),x["start"]),axis = 1 )
Ast_enh_end = Ast_enh.apply(lambda x:  lf.chrpos_liftover(x["chr"].replace("chr",""),x["end"]),axis = 1 )
Ast_enh["start"] =  [x[1] for x in Ast_enh_start]
Ast_enh["end"] =  [x[1] for x in Ast_enh_end]
Ast_enh.to_csv("//sc/arion/projects/load/users/sunh14/annotation_interaction_gwas/Ast_hg38.Enh.Nott.tsv","\t")

## Verification (Interactively in this notebook)

In [39]:
Ast_enh_lifted = pd.read_csv("/sc/arion/projects/load/users/patelt16/projects/regulatory_gwas/20230429/wgs.reg.vcf_to_gds/input/Nott2019_LHX2_H3K27ac_TableS5_prom_enh_hg38.bed",sep = "\t",header = None)

In [40]:
Ast_enh_new_lifted = pd.read_csv("//sc/arion/projects/load/users/sunh14/annotation_interaction_gwas/Ast_hg38.Enh.Nott.tsv",sep = "\t")

In [41]:
Ast_enh_lifted

,0,1,2,3
0,1,817069,817440,ENH:1:817069:817440
1,1,818117,818345,ENH:1:818117:818345
2,1,818682,818799,ENH:1:818682:818799
3,1,904119,905001,ENH:1:904119:905001
4,1,910178,910472,ENH:1:910178:910472
...,...,...,...,...
64172,Y,14841238,14843072,ENH:Y:14841238:14843072
64173,Y,15424759,15424948,ENH:Y:15424759:15424948
64174,Y,17234038,17234873,ENH:Y:17234038:17234873
64175,Y,20576971,20577183,ENH:Y:20576971:20577183


In [42]:
Ast_enh_new_lifted

,Unnamed: 0,chr,start,end
0,0,chr1,817069,817440
1,1,chr1,818117,818345
2,2,chr1,818682,818799
3,3,chr1,904119,905001
4,4,chr1,910178,910472
...,...,...,...,...
63806,63806,chrY,14841238,14843072
63807,63807,chrY,15424759,15424948
63808,63808,chrY,17234038,17234873
63809,63809,chrY,20576971,20577183


### From UCSC Browser
```
chr1	817069	817440	chr1:752450-752820	1
chr1	818117	818345	chr1:753498-753725	1
chr1	818682	818799	chr1:754063-754179	1
chr1	904119	905001	chr1:839500-840381	1
chr1	910178	910472	chr1:845559-845852	1
chr1	939920	940068	chr1:875301-875448	1
chr1	976023	976683	chr1:911404-912063	1
chr1	976912	977020	chr1:912293-912400	1
chr1	997336	997471	chr1:932717-932851	1
chr1	998014	998404	chr1:933395-933784	1
chr1	1002289	1002359	chr1:937670-937739	1
chr1	1005170	1005899	chr1:940551-941279	1
chr1	1012736	1012794	chr1:948117-948174	1
chr1	1014645	1015255	chr1:950026-950635	1
chr1	1018940	1019069	chr1:954321-954449	1
chr1	1024021	1024185	chr1:959402-959565	1
chr1	1024530	1025572	chr1:959911-960952	1
chr1	1032727	1033069	chr1:968108-968449	1
```

SyntaxError: invalid syntax (1608175822.py, line 1)